# Homework №9

## Imports

In [1]:
# !wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/transfer_learning_hg/lenta_sample.csv

In [2]:
# !pip install transformers[torch] datasets wandb torcheval -q

In [3]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torcheval.metrics import MulticlassPrecision

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.auto import trange, tqdm
import pandas as pd
import wandb

In [4]:
wandb.login()

wandb: Currently logged in as: tokubetsu01. Use `wandb login --relogin` to force relogin


True

In [5]:
wandb.init(
    project='actprob-hw9',
    group='part-1'
)

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Part 1

In [7]:
class LentaDataset(Dataset):
    def __init__(self,
                 data: pd.DataFrame,
                 le: LabelEncoder = None):

        self.data = data

        if le is None:
            self.le = LabelEncoder()
            self.le.fit(self.data['topic'])
        else:
            self.le = le

        self.data['label'] = self.le.transform(self.data['topic'])

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return (
            self.data.text[idx],
            self.data.label[idx]
        )

In [8]:
def evaluate(model,
             model_name,
             epoch,
          tokenizer,
          metric,
          data):
    with torch.no_grad():
        tq_batch = tqdm(data, leave=False)
        for batch in tq_batch:
            tok_data = tokenizer(
                batch[0],
                padding='max_length',
                truncation=True,
                max_length=seq_len,
                return_tensors='pt'
            )

            pred = model(**{k: v.to(model.device) for k, v in tok_data.items()},
                         output_hidden_states=True)

            metric.update(pred.logits, batch[1])
            tq_batch.update(1)

        acc = metric.compute()
        wandb.log({f"Eval/Epoch": epoch,
                    f'Eval/{model_name}/Accuracy': acc})

In [9]:
def train(model,
          model_name,
          tokenizer,
          optimizer,
          criterion,
          metric,
          data_train,
          data_eval,
          losses,
          n_epochs=1):

    model.train()

    for epoch in trange(n_epochs, desc='Epochs: '):
        tq_batch = tqdm(data_train, leave=False)
        for i, batch in enumerate(tq_batch):
            tok_data = tokenizer(
                batch[0],
                padding='max_length',
                truncation=True,
                max_length=seq_len,
                return_tensors='pt'
            )

            pred = model(**{k: v.to(model.device) for k, v in tok_data.items()})

            loss = criterion(pred.logits, batch[1].to(model.device))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            losses.append(loss.detach().cpu())

            loss = (sum(losses[-100:]) / len(losses[-100:])).item()
            metric.update(pred.logits, batch[1].to(model.device))
            acc = metric.compute().item()

            wandb.log({f"Train/{model_name}/Batch": i,
                       f"Train/{model_name}/Loss": loss,
                       f"Train/{model_name}/Accuracy": acc})
            tq_batch.set_postfix({
                    'Loss': loss,
                    'Accuracy': acc
                })

            tq_batch.update(1)

        evaluate(model, model_name, epoch + 1, tokenizer, metric, data_eval)
    model.eval()

In [10]:
def train_and_eval(model_name, dl_train, dl_test):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    id2label = {i: lbl for i, lbl in enumerate(dt.le.classes_)}
    label2id = {lbl: i for i, lbl in enumerate(dt.le.classes_)}
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=dt.data.label.unique().shape[0],
        id2label=id2label,
        label2id=label2id
    )
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    metric = MulticlassPrecision(num_classes=dt.data.label.unique().shape[0])

    losses = []
    evaluate(model, model_name, 0, tokenizer, metric, dl_test)
    train(model,
          model_name,
        tokenizer,
        optimizer,
        criterion,
        metric,
        dl_train,
        dl_test,
        losses,
        n_epochs=n_epochs)

    model.cpu()

In [11]:
seq_len = 128
batch_size = 32
n_epochs = 10

data = pd.read_csv('lenta_sample.csv')
data = data.dropna(subset=['topic', 'text']).reset_index(drop=True)
train_data, test_data = train_test_split(data,
                               test_size=0.05,
                               stratify=data.topic,
                               random_state=42)

dt = LentaDataset(train_data.reset_index(drop=True))
dl_train = DataLoader(dt, batch_size=batch_size, shuffle=True)

dt_test = LentaDataset(test_data.reset_index(drop=True), le=dt.le)
dl_test = DataLoader(dt_test, batch_size=batch_size, shuffle=True)

In [12]:
model_name = 'distilbert/distilbert-base-uncased'
train_and_eval(model_name, dl_train, dl_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
model_name = 'microsoft/deberta-v3-base'
train_and_eval(model_name, dl_train, dl_test)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
model_name = 'FacebookAI/xlm-roberta-base'
train_and_eval(model_name, dl_train, dl_test)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Вот тут живут результаты в wandb: https://api.wandb.ai/links/tokubetsu01/307zfny7

Но вообще, получилось не очень. Да, оно обучается чуть-чуть, но кажется, что скорее нет. Лосс падает, качество не растет. По хорошему, надо бы руками писать голову классификации, так как я не очень доверяю тому, что сделал HF, но я не успела :(

## Part 2

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

Большой гпт не влез в колаб, так что взяла маленький. По опыту, он немного туповат (особенно после чатгпт), но живем с тем, что есть.

In [5]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Текст отсюда:  https://ficbook.net/readfic/7233454

In [6]:
train_path = '/content/Poslednie-Geroi-Fikbuka.txt'
with open(train_path, "r") as f:
    text = f.read()
print(text)

Смелый Восьмиклассник
Вышел в интернет,
Дома затыкают,
А на сайтах — нет,

Можно по сети всех
Грязью поливать,
Не дадут по шее,
Как сестра и мать.

Смелый Восьмиклассник
Знает: он мужиг!
Дохуя брутален,
Скажет напрямик,

Как же не по нраву
Каждый персонаж,
И сюжет, и юмор,
В общем, фанфик ваш.

Вам глаза раскроет
Кто же, как не он?!
У него ж дымится
Новый телефон!

Накатать вам «простынь»
Жизни дело всей,
Ведь на целом свете
Нет его умней!

Смелый Восьмиклассник,
Призванный Судьбой,
Он Герой Фикбука…

Последний Герой!

Королева Драмы,
В будущем «Яжмать!»
Право заимела
Всё в лицо сказать,

У неё всех больше
Всяких разных прав,
И не смей тут вякать,
Власть её поправ!

Ведь она Читатель!
Снизошла к тебе!
Осветила знаньем
Путника во тьме!

И кусок какашки
Нищему дала…
«Что? Как так «не надо»?
Что, блин, за дела?!»

Королева Драмы
Вмиг включает МХАТ:
«Как посмел ты, смертный,
Не сказать, что рад?!

Будь мне благодарен,
Тварь ты, сука, блядь!»
В общем, тут истерик
Дней этак на пять.

Королев

In [7]:
train_dataset = TextDataset(tokenizer=tokenizer,
                            file_path=train_path,
                            block_size=64,
                            overwrite_cache=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
training_args = TrainingArguments(
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.train()

wandb: Currently logged in as: tokubetsu01. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.8944805908203124, metrics={'train_runtime': 44.7813, 'train_samples_per_second': 44.662, 'train_steps_per_second': 2.233, 'total_flos': 65323008000000.0, 'train_loss': 1.8944805908203124, 'epoch': 100.0})

In [12]:
# повтоярем параметры из лекции
text = "Большое окно "
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=150,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Большое окно 
Вниз! 

Кругом — дебилы,
А на самом деле — дураки,
Жадные до денег,

Попавшие в Книгу рекордов Гиннеса,
Скажу сразу: я не знаю,
Как вам это объяснить…

И всё тут.
Я знаю точно: этот
Последний Герой! 

Пафосный Филолог
Должен был бы быть самым
Множеством других…
Но он Один! 

У него ДВА имени,
И ни одного
Точно такого же…

(Чёрненький Завистник)
Диванный Эксперт
Обоснуй всех этих



```
Большое окно
Вниз, к звёздам!

Кругом — дураки,
А на самом деле — умницы.
Попасть в Книгу рекордов Гиннеса —
Самый большой подвиг!

И пусть не умеет Левша,
Будь он хоть трижды гений,
Он всё равно — ноль!

Ненависть и злоба
Должны быть у всех,
Но только у него!

Королева Драмы
Знает точно: лучше всех
Всех она знает…
(с) Евгений Евстигнеев

***
«Книга рекордов Гиннеса» — самая большая в мире книга рекордов по количеству прочитанных страниц. В ней собраны самые невероятные факты о жизни людей за всю историю человечества.

Оригинал взят у  в «Книга рекордов Гиннеса»
```
Он научился писать стихи, причем даже авторство им ставит. Правда Евгений Евстигнеев скорее актер, чем поэт, но гпт старался :)

In [20]:
# попробуем с температурой вместо beam_search
text = "Большое окно "
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                         temperature=0.1,
                         top_k=1,
                        max_length=150,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Большое окно 
Вниз! 
        (c) &nbsp;CinemaScene.ru&laquo;&hellip;.Я не знаю, как вам объяснить: я — человек с ограниченными возможностями и мне очень сложно их преодолеть в силу разных причин… Но у меня есть для вас небольшая инструкция по преодолению этих трудностей!»(с).Оригинал взят ув Большое окошко / Как стать счастливым?Как же быть тем счастливчиком из маленького городка на Земле?..Часть 1Счастье начинается тамГде кончается мечта...И заканчивается труд всей жизни..© Copyright © 2015 by CineBlogerВсе материалы блога можно копировать только при наличии активной ссылки on Blogs2ljusnext


С температурой он почему-то отказывается генерировать так же красиво, как с beam_search. Я сделаа попыток 20 с ращзной температурой и штрафом за повторы, но все равно неинтересно.
```
Большое окно
Вниз по наклонной,
        Вверх — не надо! —…[1 - Перевод М. Лозинского.]]* * Смело входите в лабиринт!.. [2 – Здесь и далее примечания переводчика.]. (Примечание автора.)][3 © Фразеологизм «непохож на оригинал» («The Fanatic Translation», перевод с английского).].
                           ***\t«Пишет вам читатель: я очень люблю этот текст».        «…Я знаю точно…»© Фраза из книги Стивена Кинга.[4 + Размер оригинала около 300 кб., включая все ссылки).]**/*******(Если кто-то
```

In [29]:
# теперь с beam_search, но дадим ему больше вариантов
text = "Вредная девочка "
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=20, top_k=50,
                        max_length=150,
                        repetition_penalty=5.0
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Вредная девочка 
Скажет, что «он — дебил»,
А на самом деле — нормальный.

Пафосный Филолог
Попадет в Книгу Рекордов Гиннеса,
Если будет знать,
Как писать грамотно.

Чёрненький Завистник
Должен быть Среднестатистиком,
А не Суперменом.

Завистник должен быть Общительным,
А не Тупым Жопочтителем,
А не Фиксированным Читателем.

Общительный Читатель — это Герой Социалистического Труда,
Филолог — Гражданин Страны Восходящего Солнца,


Очень глупый пример, который одновременно очень в стиле того, на чем я его дообучала:
```
Вредная девочка
Скажет, что «он — дебил»,
А на самом деле — нормальный.

Пафосный Филолог
Попадет в Книгу Рекордов Гиннеса,
Если будет знать,
Как писать грамотно.

Чёрненький Завистник
Должен быть Среднестатистиком,
А не Суперменом.

Завистник должен быть Общительным,
А не Тупым Жопочтителем,
А не Фиксированным Читателем.

Общительный Читатель — это Герой Социалистического Труда,
Филолог — Гражданин Страны Восходящего Солнца,
```

In [31]:
# все еще с beam_search, но не будем больше ничего указывать
text = "Вредная девочка "
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        max_length=150,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Вредная девочка 
Скажет, что ей 
Вредят 
Все, что под руку попадется,

Будь она неладна,
Которая в друзьях,
А не в друзьях!

Вредная девочка 
Скажет, что ей 
Вредят 
Все, что под руку попадется,

Будь она неладна,
Которая в друзьях,
А не в друзьях!

Вредная девочка 
Скажет, что ей 
Вредят 
Все, что под руку попадется,

Будь она неладна,
Которая в друзьях,


Прикольно, но дублируется. Штраф за повторы надо оставлять в любом случае, судя по всему.

In [33]:
# все еще с beam_search, но не будем больше ничего указывать
text = "Вредная девочка "
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=2,
                        max_length=150,
                         repetition_penalty=5.0,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Вредная девочка 
Жизни не знает…

Правоохранительным органам 

Что делать, если в туалете
Унитаз сломался?
А может, это у вас
Тряпка какая-то?

Вызовите «скорую»
И срочно к врачу!
Диагноз поставил страшный
Пациент с жалобами
На проблемы со здоровьем.

Срочно к доктору!
Лекарства без рецепта
Не под силу ни одному —
Вот и приходится
Вам вызванивать
«скорую помощь».

Если вдруг что-то пошло не так,
К вам на помощь придёт
Специалист по диагностике
Здоровья вашей


Наверное, мне вот это
```
Вредная девочка
Ждет тебя весь день!

Скажет, что устала,
Что дела нет совсем,
Что на работе не в кайф,
И вообще…
Все будет как надо,
Главное – результат!

Ведь это ты ее научил
Самому простому —
Не паниковать и не унывать.
Ты ей помог,
А теперь сам должен
Тебе помочь!»

Вот так она его простила,
У него все получилось,
Он понял, что зря
Путал их имена,
Совсем чуть-чуть…

Но вот беда — он забыл
Названия всех книг,
Словно прочитал их с трудом,
```
нравится больше всего. Тут даже как будто есть какой-то сюжет, хотя и не очень понятный...